In [2]:
import requests

def send_motor_command(option):
    url = "http://192.168.100.150/sendOption"  # Địa chỉ IP của ESP8266
    data = {'option': option}  # Dữ liệu gửi đi, là tùy chọn bật/tắt động cơ

    try:
        # Gửi yêu cầu POST đến ESP8266
        response = requests.post(url, data=data)
        # Kiểm tra phản hồi từ server
        if response.status_code == 200:
            print("Gửi thành công, Động cơ đã được điều khiển theo tùy chọn:", option)
        else:
            print("Có lỗi xảy ra khi gửi yêu cầu:", response.status_code)
    except requests.exceptions.RequestException as e:
        print("Không thể kết nối đến ESP8266:", e)

# Ví dụ gọi hàm để bật động cơ (option = 1)

# Ví dụ gọi hàm để tắt động cơ (option = 2)
# send_motor_command(2)

In [26]:
send_motor_command(1)

Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 1


In [28]:
send_motor_command(2)

Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 2


In [31]:
send_motor_command(3)

Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 3


In [32]:
send_motor_command(4)

Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 4


# v1

In [ ]:
// C:\Users\ADMIN\Documents\Arduino 
// ĐỘNG CƠ DC
#include <ESP8266WiFi.h>
#include <ESP8266WebServer.h>

// Khai báo các chân điều khiển động cơ và LED
const int motorPin1 = D6; // Chân IN1 của mạch điều khiển động cơ
const int motorPin2 = D5; // Chân IN2 của mạch điều khiển động cơ
const int ledPin = D1;    // Chân kết nối LED // chân còn lại nối với điện trở sau đó nối với GND 
// 2 Chân của led cx phải cắm đúng thì mới sáng, không sáng thì đảo lại


ESP8266WebServer server(80); // Khởi tạo server trên cổng 80

void setup() {
  Serial.begin(9600);
  delay(1000);

  // Kết nối WiFi
  WiFi.begin("Day Sau", "0971404372"); // SSID và mật khẩu
  while (WiFi.waitForConnectResult() != WL_CONNECTED) {
    Serial.println("Connecting...");
  }
  Serial.println("IP: ");
  Serial.println(WiFi.localIP());

  // Thiết lập các chân điều khiển động cơ và LED là output
  pinMode(motorPin1, OUTPUT);
  pinMode(motorPin2, OUTPUT);
  pinMode(ledPin, OUTPUT);

  // Thiết lập server
  server.on("/", [] {
    server.send(200, "text/html",
                "<!DOCTYPE html>"
                "<html>"
                "<head>"
                "<title>Control DC Motor and LED</title>"
                "<script>"
                "function sendOption() {"
                "var optionInput = document.getElementById('option');"
                "var xhttp = new XMLHttpRequest();"
                "var option = optionInput.value;"
                "xhttp.open('POST', 'sendOption', true);"
                "xhttp.setRequestHeader('Content-type', 'application/x-www-form-urlencoded');"
                "xhttp.send('option=' + option);"
                "}"
                "</script>"
                "</head>"
                "<body>"
                "<label>Option (1: Bật Động Cơ, 2: Tắt Động Cơ, 3: Bật LED, 4: Tắt LED): </label>"
                "<input type='text' id='option'>"
                "<input type='button' value='Send' onclick='sendOption()'>"
                "</body>"
                "</html>");
  });

  setupRoutes(); // Thiết lập các route của server
  server.begin(); // Khởi động server
}

void handleSendOption() {
  if (server.hasArg("option")) {
    int option = server.arg("option").toInt();
    if (option == 1) {
      // Bật động cơ
      digitalWrite(motorPin1, HIGH);
      digitalWrite(motorPin2, LOW);
      Serial.println("Motor ON");
      server.send(200, "text/plain", "Motor turned ON");
    } else if (option == 2) {
      // Tắt động cơ
      digitalWrite(motorPin1, LOW);
      digitalWrite(motorPin2, LOW);
      Serial.println("Motor OFF");
      server.send(200, "text/plain", "Motor turned OFF");
    } else if (option == 3) {
      // Bật LED
      digitalWrite(ledPin, HIGH);
      Serial.println("LED ON");
      server.send(200, "text/plain", "LED turned ON");
    } else if (option == 4) {
      // Tắt LED
      digitalWrite(ledPin, LOW);
      Serial.println("LED OFF");
      server.send(200, "text/plain", "LED turned OFF");
    } else {
      server.send(400, "text/plain", "Invalid option");
    }
  } else {
    server.send(400, "text/plain", "Missing 'option' parameter");
  }
}

void setupRoutes() {
  server.on("/sendOption", handleSendOption); // Route để xử lý tùy chọn điều khiển
}

void loop() {
  server.handleClient(); // Xử lý các yêu cầu từ client
}
